# Kelompok 11

#ALRINGGA RIZKY WIDANGSA (17523045)
#MUHAMMAD IRFAN (17523149)
#ADINDA WELLDAN AL IRSYAD (17523175)

# Convolutional Neural Network dengan PyTorch

### Dataset

Silakan upload cnn_grc.zip ke files.

Alternatif lain, bisa juga dataset di upload ke storage lain lalu nanti didownload dengan cmommand lin

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Copy File dari Google Drive

In [3]:
!cp "/content/drive/Shareddrives/Deep Learning Tubes/cnn_grc.zip" "/content/"

In [4]:
# extract file zip
# tanda seru artinya ini bukan perintah Python, melainkan perintah command prompt ubuntu
! unzip cnn_grc.zip

Archive:  cnn_grc.zip
   creating: cnn_grc/
  inflating: cnn_grc/SATE0151.png    
   creating: cnn_grc/test/
   creating: cnn_grc/test/BAKSO/
  inflating: cnn_grc/test/BAKSO/BAKSO0001.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0002.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0003.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0004.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0005.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0006.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0007.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0008.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0009.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0010.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0011.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0012.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0013.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0014.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0015.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0016.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0017.png  
  inflating: cnn_grc/test/BAKSO/BAKSO0018.

### Packages

In [5]:
import torch as pt # library utama pytorch
import numpy as np # numerical python
import pandas as pd # bermain dataset
import copy # untuk copy2 an

# libray lain pytorch untuk membangun arsitektur
import torch.nn as nn 
import torch.nn.functional as F

# library dar pytorch untuk pengelolaan dataset khususnya data citra
from torchvision import transforms, datasets, models

### Preprocessing
Pada bagian ini kita akan memperbaiki data agar enak dibaca CNN

In [6]:
train_dir = "/content/cnn_grc/train"
test_dir = "/content/cnn_grc/test"

# fungsi transform bertugas untuk "mengedit" data yang masuk
# macam-macam perintah yang bisa dilakukan bisa cek di sini: https://pytorch.org/docs/stable/torchvision/transforms.html

train_transform = transforms.Compose([ # proses transform dilakukan secara berurutan
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.Resize(500),
    transforms.CenterCrop(224), # 1. di crop ukuran 150x150 di tengah citra
    transforms.ToTensor(), # 2. di ubah ke tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(500),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load dataset dengan format folder yang sesuai lalu aplikasikan transformasi ke data yang dibaca
# hasil dari proses ini adalah sebuah list berelemen tuple (tensor_citra, label) disebut dengan objek datasets
train_img = datasets.ImageFolder(train_dir, transform=train_transform)
test_img = datasets.ImageFolder(test_dir, transform=test_transform)

In [7]:
# bagian ini intinya mengubah objek datasets di atas ke format yang lebih baik :)
# insyaAllah akan dijelaskan di pertemuan selanjutnya
trainloaders = pt.utils.data.DataLoader(train_img, batch_size=16, shuffle=True)
testloaders = pt.utils.data.DataLoader(test_img, batch_size=1, shuffle=True)

In [8]:
# Informasi dari dataset kita
print("train_img type   :",type(train_img)) # objek datasets

print("train_img length :",len(train_img)) # 515 data train
print("test_img length :",len(test_img)) # 66 data test

print("train_img classes:",train_img.classes) # nama kelas
print("train_img classes:",train_img.class_to_idx) # nama kelas

print("train_img[0] type:",type(train_img[0])) # tuple (A, B)
print("train_img[0][0]  :",train_img[0][0].size()) # tensor citra
print("train_img[0][1]  :",train_img[0][1]) # label

# variabel banyak data
len_train = len(train_img)
len_test = len(test_img)

train_img type   : torchvision.datasets.folder.ImageFolder
train_img length : 827
test_img length : 300
train_img classes: ['BAKSO', 'MIEAYAM', 'SATE']
train_img classes: {'BAKSO': 0, 'MIEAYAM': 1, 'SATE': 2}
train_img[0] type: <class 'tuple'>
train_img[0][0]  : torch.Size([3, 224, 224])
train_img[0][1]  : 0


### Buat Arsitektur

In [9]:
class Net(nn.Module):
   
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        # ukuran sisi = ((ukuran - kernel_size + 2 * padding) / stride + 1)
        # hasil flatten = sisi * sisi * feature_maps
        self.fc1 = nn.Linear(42632, 3) 
    
    def forward(self, x):
        # forward propagation
        x = self.conv1(x) # convolusi
        x = F.relu(x) # fungsi aktivasi
        x = self.pool(x) # pooling
        x = x.reshape(x.shape[0], -1) # flattening
        x = F.relu(self.fc1(x)) # layer biasa
        return x

In [10]:
modelkita = Net().cuda()

In [11]:
modelkita

Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=42632, out_features=3, bias=True)
)

In [12]:
modelkita = models.resnet18(pretrained=True)

#freezing
for param in modelkita.parameters():
    param.requires_grad = False

# hapus layer terakhir
modelkita.fc = nn.Linear(512, 3)

modelkita = modelkita.cuda()
print(modelkita)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

In [13]:
optimizer = pt.optim.SGD(modelkita.parameters(), lr=0.01) # algoritma training
criterion = nn.CrossEntropyLoss() # rumus error

### Mulai Training

In [14]:
modelkita.train()
for i in range(30): # 30 epoch

    rata2error = 0 # untuk menghitung error ("seberapa salah")
    jumlahbenar = 0 # untuk menghitung akurasi ("seberapa benar")

    for image, label in trainloaders: # iterasi semua data
        
        image = image.cuda()
        label = label.cuda()        

        # forward
        Y = modelkita( image )
        E = criterion( Y, label )
        rata2error += E

        prediksi = pt.max(Y, dim=1).indices
        for i in range(len(prediksi)):
            if prediksi[i]==label[i]:
                jumlahbenar+=1
        
        # backward
        optimizer.zero_grad()
        E.backward()       

        # update
        optimizer.step() 

    print(rata2error/len_train, jumlahbenar/len_train)

tensor(0.0459, device='cuda:0', grad_fn=<DivBackward0>) 0.6928657799274486
tensor(0.0309, device='cuda:0', grad_fn=<DivBackward0>) 0.8125755743651754
tensor(0.0246, device='cuda:0', grad_fn=<DivBackward0>) 0.8524788391777509
tensor(0.0231, device='cuda:0', grad_fn=<DivBackward0>) 0.8561064087061668
tensor(0.0223, device='cuda:0', grad_fn=<DivBackward0>) 0.8730350665054414
tensor(0.0204, device='cuda:0', grad_fn=<DivBackward0>) 0.8827085852478839
tensor(0.0202, device='cuda:0', grad_fn=<DivBackward0>) 0.8827085852478839
tensor(0.0196, device='cuda:0', grad_fn=<DivBackward0>) 0.905683192261185
tensor(0.0185, device='cuda:0', grad_fn=<DivBackward0>) 0.8851269649334945
tensor(0.0199, device='cuda:0', grad_fn=<DivBackward0>) 0.8887545344619106
tensor(0.0177, device='cuda:0', grad_fn=<DivBackward0>) 0.8923821039903265
tensor(0.0189, device='cuda:0', grad_fn=<DivBackward0>) 0.8948004836759371
tensor(0.0192, device='cuda:0', grad_fn=<DivBackward0>) 0.8935912938331319
tensor(0.0190, device='cud

### Testing

In [15]:
modelkita.eval()

with pt.no_grad():
    rata2error = 0 # untuk menghitung error ("seberapa salah")
    jumlahbenar = 0 # untuk menghitung akurasi ("seberapa benar")

    for image, label in testloaders: # iterasi semua data

        image = image.cuda()
        label = label.cuda() 
       
        # forward
        Y = modelkita( image )
        E = criterion( Y, label )
        rata2error += E

        prediksi = pt.max(Y, dim=1).indices
        for i in range(len(prediksi)):
            if prediksi[i]==label[i]:
                jumlahbenar+=1

    print(rata2error/len_test, jumlahbenar/len_test)

tensor(0.1584, device='cuda:0') 0.9633333333333334


### Saving Model

In [17]:
pt.save(modelkita, "modelkita_v2.pth")

### Inference

In [18]:
# load model
modelbaru = pt.load("modelkita_v2.pth", map_location=pt.device("cpu"))

In [19]:
from PIL import Image

modelbaru.eval()

# load single image
single_img = Image.open("/content/cnn_grc/train/SATE/SATE0088.png")

# transform
single_img_transformed = test_transform(single_img)

Y = modelbaru( single_img_transformed.unsqueeze(0) )

prediksi = pt.max(Y, dim=1).indices
print(prediksi)

tensor([2])


In [20]:
prediksi.item()

2